![](https://thumbor.forbes.com/thumbor/960x0/https%3A%2F%2Fspecials-images.forbesimg.com%2Fdam%2Fimageserve%2F1027476298%2F960x0.jpg%3Ffit%3Dscale)

# Introduction:
Let us start by asking ourself a Simple question: Who is a Philosopher and what is Philosophy ?

A **Philosopher** is some who practices **Philosopy**. The term **Philosophy** is derived from the Ancient Greek word **Philosophos** meaning **Lover of Wisdom**, The coining of this term may be attributed to the great Greek Philosopher **Pythagoras** in 6th centyary B.C.E .

In this notebook we would like to Introduce you to some of the world reknowned philosophers and to carry out a Sentiment analysis on what they have said. There are 30 Philosophers in the given data set, so without further delay lets begin...

In [ ]:
# importing required libraries
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sb
import re
import tqdm
from time import perf_counter

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print( 'input  => ' + os.path.join(dirname, filename))

In [ ]:
# loading data
dataFrame = pd.read_csv('/kaggle/input/history-of-philosophy/phil_nlp.csv')
dataFrame[0:10]

Now we are writting a class for text Summarization (copying this class from one of my Previous Notebooks)
If interested you can check it here:
https://www.kaggle.com/vanvalkenberg/text-summarization-using-spacy

The basic idea behind this algorithm is to find the sentence that has the maximum number of the most commonly used words in the text corpus after removing the stopwords (like the, is, that, etc). The class I believe is self-explanatory. 

In [ ]:
# Making some essential imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
stopwords = list (STOP_WORDS)
punctuation += '\n'
from heapq import nlargest


# Defining a Class for extracting Summary
class Summarizer():
    
    ## constructor for data loading
    ## give either path or corpus itself
    def __init__(self, path = None, corpus = None):
        self.mCorpus = ''
        self.mWordFrequencies = {}
        self.mSentScore = {}
        self.mNumSentences = 0
        
        if path is not None:
            file = open(path, "r")
            self.mCorpus = self.mCorpus + file.read()
        elif corpus is not None:
            self.mCorpus = corpus
            
        self.mNlp = spacy.load('en_core_web_sm')
        self.mDoc = self.mNlp(self.mCorpus)
    
    
    # Function for Printing Corpus
    def PrintCorpus(self):
        print(self.mCorpus)
    
    
    ## defining a function for Calculating 
    ## word frequiences
    def WordFrequencyCalculator(self):
        for word in self.mDoc:
            wordInLowerCase = word.text.lower()
            
            if (wordInLowerCase not in stopwords) and (wordInLowerCase not in punctuation):
                if wordInLowerCase not in self.mWordFrequencies.keys():
                    self.mWordFrequencies[wordInLowerCase] = 1
                else:
                    self.mWordFrequencies[wordInLowerCase] += 1
    
        return
    
    
    ## Function for normalizing Frequency Values
    def WordFreqNormalizer(self):
        maxFreq = max(self.mWordFrequencies.values())
        
        for word in self.mWordFrequencies.keys():
            self.mWordFrequencies[word] = self.mWordFrequencies[word] / maxFreq
        
        return
    
    # Function For calculating  sentence score
    # based on computed normalized word Frequencies
    def CalSentScore(self):
        sentences = [sent for sent in self.mDoc.sents]
        self.mNumSentences = len(sentences)
        
        for sent in sentences:
            for word in sent:
                wordInLowerCase = word.text.lower()
                
                if wordInLowerCase in self.mWordFrequencies.keys():
                    if sent not in self.mSentScore.keys():
                        self.mSentScore[sent] = self.mWordFrequencies[wordInLowerCase]
                    else:
                        self.mSentScore[sent] += self.mWordFrequencies[wordInLowerCase]
        return
    
    ## Writting the MainFunction for this script now
    def SummarizeMyText(self, fractionToReduce = 0.2):
        self.WordFrequencyCalculator()
        self.WordFreqNormalizer()
        self.CalSentScore()
        
        reducedSentNum = int (self.mNumSentences * fractionToReduce)
        print ('Total number of Sentences = {}'.format(self.mNumSentences))
        print('Num of Sentences Reduced to {}'.format(reducedSentNum))
        print('Summary as follows : \n')
        
        summaryList = nlargest(reducedSentNum, self.mSentScore, key = self.mSentScore.get)
        for sent in summaryList:
            print(sent, end = '')

# A Function Is All you Need!

In [ ]:
# World Cloud Plotter
from wordcloud import WordCloud, STOPWORDS 
stopwords = set(STOPWORDS) 

def WordCloudForSentiments (corpus, title):
    wordcloud = WordCloud(width = 800, height = 800,background_color ='grey',
                          stopwords = stopwords,  min_font_size = 10).generate(corpus)
    
    plt.figure(figsize = (9, 9), facecolor = None) 
    plt.imshow(wordcloud) 
    plt.rcParams.update({'font.size': 25})
    plt.axis("off") 
    plt.title('Word Cloud:  ' + title)
    plt.tight_layout(pad = 0) 
  
    plt.show() 

# A Function for Sentiment Analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer
def SentimentAnlysis(sentence):
    sentAnalyzer = SentimentIntensityAnalyzer() 
    sentDict = sentAnalyzer.polarity_scores(sentence)
    
    if sentDict['compound'] >= 0.05:
        return "positive"
    elif sentDict['compound'] <= -0.05 :
        return "negative"
    else:
        return "neutral"


# Our Main Function
def Analyzer(dataFrame, author):
    start = perf_counter()
    df = dataFrame [dataFrame['author'] == author] 
        
    #making a Corpus and finding sentiments
    corpus = ''
    numPostives = 0
    numNegatives = 0
    numNeutrals = 0
    
    for mem in df['sentence_lowered']:
        corpus += mem
    
    for i in range (len(df)):
        sent = (SentimentAnlysis(df['sentence_lowered'].iloc[i]))
        if sent == "positive":
            numPostives += 1
        elif sent == "negative":
            numNegatives += 1
        else:
            numNeutrals += 1
    
    plt.figure(figsize = (7, 7))
    plt.pie([numPostives, numNegatives, numNeutrals], labels = ['positives', 'negatives', 'neutrals'], autopct='%1.2f%%')
    plt.title('Sentiment Analysis for  Philosopher: ' + author)
   
    #text Cleaning and plotting wordCloud
    corpus = re.sub("[^a-zA-Z0-9]+", ' ', corpus)
    WordCloudForSentiments(corpus,'WordCloud for  Philosopher: ' + author )
    
    # Extractive Text summarization
    print ('The following lines represent\'s what the author said: ')
    summarizer = Summarizer(corpus = corpus[:1000000] )
    summarizer.SummarizeMyText(fractionToReduce = 0.002)
    print('\n')
    
    # Benchmarks
    print('Time taken: {}'.format(perf_counter() - start ))
    

# Aristortle
![](https://upload.wikimedia.org/wikipedia/commons/thumb/a/ae/Aristotle_Altemps_Inv8575.jpg/330px-Aristotle_Altemps_Inv8575.jpg)

Aristortle was born around 384 B.C.E in the Greek kingdom of Macedonia, where his father was the royal doctor. He grew up to be most influencial Philosopher ever and was given nicknames like **The Master** and **The Philosopher**, Oh forgot to mention he was also the tutor of **Alexander The Great**. There are 4 great questions which Aristortle has answered:

*  What makes people Happy ?

Hmm, So According to Aristortle a person his happy if he/she posses the following virtues: Courage, Temperance, Liberality, Magnificence, Pride, Patience, Truthfulness, Witiness, Friendliness, Modesty.

* What is Art For?

Aristortle has written a manuel for writting great plays named **The Poetics**. He has introduced the ideas of **PeriPeteia** or sudden change in fornute and **Anagnoresis** or a sudden moment of dramatic revelation. By these literary devices Aristorle wants to bring home the notion that tragedy can befall on anyone, and in such moments we must be kind and compasenate toward others.

* What are Friends for ?
According to Aristortle there are 3 kinds of friendships:

 * The one where each individual is seeking fun or him/herself.
 * The one where everyone wants a strategic advantage from one another.
 * The one which is True Friendship.


* How can ideas cut through a Busy World ?

Like most of us, Aristortle also wondered why the Best idea not always wins the argument? He studied his surroundings and came up with the art of **Rhetoric** where he emphasied on the ideas of making a discussion funny (because people has short attention spans), being empathetic toward other person and  being persuasive.

(reference: https://www.youtube.com/watch?v=csIW4W_DYX4)

In [ ]:
Analyzer(dataFrame, 'Aristotle')

# Berkeley
![](https://upload.wikimedia.org/wikipedia/commons/thumb/5/5d/John_Smibert_-_Bishop_George_Berkeley_-_Google_Art_Project.jpg/330px-John_Smibert_-_Bishop_George_Berkeley_-_Google_Art_Project.jpg)

George Berkeley (12 March 1685 – 14 January 1753) – known as Bishop Berkeley (Bishop of Cloyne of the Anglican Church of Ireland) – was an Anglo-Irish philosopher whose primary achievement was the advancement of a theory he called "immaterialism" (later referred to as "subjective idealism" by others). This theory denies the existence of material substance and instead contends that familiar objects like tables and chairs are ideas perceived by the minds and, as a result, cannot exist without being perceived. Berkeley is also known for his critique of abstraction, an important premise in his argument for immaterialism.

In [ ]:
Analyzer(dataFrame, 'Berkeley')

# Deleuze
![](https://upload.wikimedia.org/wikipedia/en/d/de/Gilles_Deleuze.jpg)

Gilles Deleuze (18 January 1925 – 4 November 1995) was a French philosopher who, from the early 1950s until his death in 1995, wrote on philosophy, literature, film, and fine art. His most popular works were the two volumes of Capitalism and Schizophrenia: Anti-Oedipus (1972) and A Thousand Plateaus (1980), both co-written with psychoanalyst Félix Guattari. His metaphysical treatise Difference and Repetition (1968) is considered by many scholars to be his magnum opus. An important part of Deleuze's oeuvre is devoted to the reading of other philosophers: the Stoics, Leibniz, Hume, Kant, Nietzsche, and Bergson, with particular influence derived from Spinoza.A. W. Moore, citing Bernard Williams's criteria for a great thinker, ranks Deleuze among the "greatest philosophers". Although he once characterized himself as a "pure metaphysician",[16] his work has influenced a variety of disciplines across the humanities, including philosophy, art, and literary theory, as well as movements such as post-structuralism and postmodernism.

In [ ]:
Analyzer(dataFrame, 'Deleuze')

# Derrida

![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/9c/Derrida_Dibujo.jpg/330px-Derrida_Dibujo.jpg)

Jacques Derrida (July 15, 1930 – October 9, 2004) was a French philosopher best known for developing a form of semiotic analysis known as deconstruction, which he analyzed in numerous texts, and developed in the context of phenomenology.He is one of the major figures associated with post-structuralism and postmodern philosophy.

During his career Derrida published more than 40 books, together with hundreds of essays and public presentations. He had a significant influence on the humanities and social sciences, including philosophy, literature, law, anthropology,historiography, applied linguistics, sociolinguistics, psychoanalysis and political theory.

His work retains major academic influence throughout the US continental Europe, South America and all other countries where continental philosophy has been predominant, particularly in debates around ontology, epistemology (especially concerning social sciences), ethics, aesthetics, hermeneutics, and the philosophy of language. In most of the Anglosphere, where analytic philosophy is dominant, Derrida's influence is most presently felt in literary studies due to his longstanding interest in language and his association with prominent literary critics from his time at Yale. He also influenced architecture (in the form of deconstructivism), music,art, and art criticism.

Particularly in his later writings, Derrida addressed ethical and political themes in his work. Some critics consider Speech and Phenomena (1967) to be his most important work. Others cite: Of Grammatology (1967), Writing and Difference (1967), and Margins of Philosophy (1972). These writings influenced various activists and political movements. He became a well-known and influential public figure, while his approach to philosophy and the notorious abstruseness of his work made him controversial.

In [ ]:
Analyzer(dataFrame, 'Derrida')

# Descartes

![](https://upload.wikimedia.org/wikipedia/commons/thumb/7/73/Frans_Hals_-_Portret_van_Ren%C3%A9_Descartes.jpg/330px-Frans_Hals_-_Portret_van_Ren%C3%A9_Descartes.jpg)

René Descartes (31 March 1596 – 11 February 1650) was a French-born philosopher, mathematician, and scientist who spent a large portion of his working life in the Dutch Republic, initially serving the Dutch States Army of Maurice of Nassau, Prince of Orange and the Stadtholder of the United Provinces. One of the most notable intellectual figures of the Dutch Golden Age,Descartes is also widely regarded as one of the founders of modern philosophy.

Many elements of Descartes's philosophy have precedents in late Aristotelianism, the revived Stoicism of the 16th century, or in earlier philosophers like Augustine. In his natural philosophy, he differed from the schools on two major points: first, he rejected the splitting of corporeal substance into matter and form; second, he rejected any appeal to final ends, divine or natural, in explaining natural phenomena. In his theology, he insists on the absolute freedom of God's act of creation. Refusing to accept the authority of previous philosophers, Descartes frequently set his views apart from the philosophers who preceded him. In the opening section of the Passions of the Soul, an early modern treatise on emotions, Descartes goes so far as to assert that he will write on this topic "as if no one had written on these matters before." His best known philosophical statement is "cogito, ergo sum" ("I think, therefore I am"; French: Je pense, donc je suis), found in Discourse on the Method (1637; in French and Latin) and Principles of Philosophy (1644, in Latin).

Descartes has often been called the father of modern philosophy, and is largely seen as responsible for the increased attention given to epistemology in the 17th century.He laid the foundation for 17th-century continental rationalism, later advocated by Spinoza and Leibniz, and was later opposed by the empiricist school of thought consisting of Hobbes, Locke, Berkeley, and Hume. In the 17th-century Dutch Republic, the rise of early modern rationalism – as a highly systematic school of philosophy in its own right for the first time in history – exerted an immense and profound influence on modern Western thought in general, with the birth of two influential rationalistic philosophical systems of Descartes (who spent most of his adult life and wrote all his major work in the United Provinces of the Netherlands) and Spinoza – namely Cartesianism and Spinozism. It was the 17th-century arch-rationalists like Descartes, Spinoza and Leibniz who have given the "Age of Reason" its name and place in history. Leibniz, Spinoza, and Descartes were all well-versed in mathematics as well as philosophy, and Descartes and Leibniz contributed greatly to science as well.

In [ ]:
Analyzer(dataFrame, 'Descartes')

# Fichte
![](https://upload.wikimedia.org/wikipedia/commons/8/8d/Johann_Gottlieb_Fichte.jpg)

Johann Gottlieb Fichte (19 May 1762 – 29 January 1814) was a German philosopher who became a founding figure of the philosophical movement known as German idealism, which developed from the theoretical and ethical writings of Immanuel Kant. Recently, philosophers and scholars have begun to appreciate Fichte as an important philosopher in his own right due to his original insights into the nature of self-consciousness or self-awareness. Fichte was also the originator of thesis–antithesis–synthesis, an idea that is often erroneously attributed to Hegel.Like Descartes and Kant before him, Fichte was motivated by the problem of subjectivity and consciousness. Fichte also wrote works of political philosophy; he has a reputation as one of the fathers of German nationalism.

In [ ]:
Analyzer(dataFrame, 'Fichte')

# Foucault
![](https://upload.wikimedia.org/wikipedia/commons/7/76/Michel_Foucault_1974_Brasil.jpg)

Paul-Michel Doria Foucault (15 October 1926 – 25 June 1984) was a French philosopher, historian of ideas, writer, political activist, and literary critic.

Foucault's theories primarily address the relationship between power and knowledge, and how they are used as a form of social control through societal institutions. Though often cited as a structuralist and postmodernist, Foucault rejected these labels. His thought has influenced academics, especially those working in communication studies, anthropology, psychology, sociology, criminology, cultural studies, literary theory, feminism, Marxism and critical theory.

Born in Poitiers, France, into an upper-middle-class family, Foucault was educated at the Lycée Henri-IV, at the École Normale Supérieure, where he developed an interest in philosophy and came under the influence of his tutors Jean Hyppolite and Louis Althusser, and at the University of Paris (Sorbonne), where he earned degrees in philosophy and psychology. After several years as a cultural diplomat abroad, he returned to France and published his first major book, The History of Madness (1961). After obtaining work between 1960 and 1966 at the University of Clermont-Ferrand, he produced The Birth of the Clinic (1963) and The Order of Things (1966), publications which displayed his increasing involvement with structuralism, from which he later distanced himself. These first three histories exemplified a historiographical technique Foucault was developing called "archaeology."

From 1966 to 1968, Foucault lectured at the University of Tunis before returning to France, where he became head of the philosophy department at the new experimental university of Paris VIII. Foucault subsequently published The Archaeology of Knowledge (1969). In 1970, Foucault was admitted to the Collège de France, a membership he retained until his death. He also became active in a number of left-wing groups involved in campaigns against racism and human rights abuses and for penal reform. Foucault later published Discipline and Punish (1975) and The History of Sexuality (1976), in which he developed archaeological and genealogical methods which emphasized the role that power plays in society.

Foucault died in Paris from complications of HIV/AIDS; he became the first public figure in France to die from complications of the disease. His partner Daniel Defert founded the AIDES charity in his memory.

In [ ]:
Analyzer(dataFrame, 'Foucault')

# Hegel

![](https://upload.wikimedia.org/wikipedia/commons/thumb/c/ce/Hegel_by_Schlesinger.jpg/330px-Hegel_by_Schlesinger.jpg)

Georg Wilhelm Friedrich Hegel (27 August 1770 – 14 November 1831) was a German philosopher, and the most important figure in German idealism. He is considered one of the fundamental figures of modern Western philosophy, with his influence extending to the entire range of contemporary philosophical issues, from aesthetics to ontology to politics, both in the analytic and continental tradition.

Hegel's principal achievement was his development of a distinctive articulation of idealism, sometimes termed absolute idealism, in which the dualisms of, for instance, mind and nature and subject and object are overcome. His philosophy of spirit conceptually integrates psychology, the state, history, art, religion and philosophy. His master–slave dialectic has been influential, especially in 20th-century France. Of special importance is his concept of spirit (Geist, sometimes also translated as "mind") as the historical manifestation of the logical concept – and the "sublation" (Aufhebung, integration without elimination or reduction) – of seemingly contradictory or opposing factors: examples include the apparent opposition between necessity and freedom and between immanence and transcendence. (Hegel has been seen in the twentieth century as the originator of the thesis, antithesis, synthesis triad, but as an explicit phrase it originated with Johann Gottlieb Fichte.)

Hegel has influenced many thinkers and writers whose own positions vary widely. For example, "the roots of post-structuralism and its unifying basis lies, in large part, in a general opposition not to the philosophical tradition tout court but specifically to the Hegelian tradition" dominating philosophy in the twentieth century prior to post-structuralism. Paul Tillich wrote that the historical dialectical thought of Hegel "has influenced world history more profoundly than any other structural analysis." Karl Barth described Hegel as a "Protestant Aquinas" while Maurice Merleau-Ponty wrote that "all the great philosophical ideas of the past century—the philosophies of Marx and Nietzsche, phenomenology, German existentialism, and psychoanalysis—had their beginnings in Hegel."

Hegel's work has been considered the "completion of philosophy" by multiple of the most influential thinkers in existentialism, post-structuralism, and twentieth-century theology. Derrida wrote of Hegel in his work Of Grammatology that "he undoubtedly summed up the entire philosophy of the logos. He determined ontology as absolute logic; he assembled all the delimitations of philosophy as presence," later remarking that Hegel is thus "the last philosopher of the book and the first philosopher of writing," indicating the relation of Hegel to post-structural thought by stating "if there were a definition of Différance, it would be precisely the limit, the interruption, the destruction of the Hegelian dialectical synthesis wherever it operates."In his work Systematic Theology, theologian Paul Tillich referred to Hegel's work as "perfect essentialism," later writing "essentialism was in Hegel's system fulfilled." Martin Heidegger observed in his 1969 work Identity and Difference and in his personal Black Notebooks that Hegel's system in an important respect "consummates western philosophy"by completing the idea of the logos, the self-grounding ground, in thinking through the identification of Being and beings, which is "the theme of logic", writing "It is... incontestable that Hegel, faithful to tradition, sees the matter of thinking in beings as such and as a whole, in the movement of Being from its emptiness to its developed fullness." Heidegger in various places further stated Hegel's thinking to be "the most powerful thinking of modern times."

In [ ]:
Analyzer(dataFrame, 'Hegel')

# Heidegger

![](https://upload.wikimedia.org/wikipedia/commons/thumb/6/64/Heidegger_2_%281960%29.jpg/330px-Heidegger_2_%281960%29.jpg)

Martin Heidegger (26 September 1889 – 26 May 1976) was a German philosopher who is widely regarded as one of the most important philosophers of the 20th century. He is best known for contributions to phenomenology, hermeneutics, and existentialism.

In Being and Time (1927), Heidegger addresses the question of the meaning of "being", which "is concerned with what it is that makes beings intelligible as beings".Heidegger approaches this question through an analysis of Dasein, his term for the specific type of being that humans possess.The reason for this approach is that we already have a pre-ontological understanding of being that shapes how we experience the world. This conception of the human is in contrast with that of Rationalist thinkers like René Descartes, who had understood human existence most basically as thinking, as in Cogito ergo sum ("I think therefore I am").

Heidegger's later work includes criticism of the view, common in the Western tradition, that all of nature is a "standing reserve" on call, as if it were a part of industrial inventory.

In [ ]:
Analyzer(dataFrame, 'Heidegger')

# Hume

![](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Painting_of_David_Hume.jpg/330px-Painting_of_David_Hume.jpg)

David Hume ((26 April 1711 OS) – 25 August 1776) was a Scottish Enlightenment philosopher, historian, economist, librarian and essayist, who is best known today for his highly influential system of philosophical empiricism, skepticism, and naturalism.Beginning with A Treatise of Human Nature (1739–40), Hume strove to create a naturalistic science of man that examined the psychological basis of human nature. Hume argued against the existence of innate ideas, positing that all human knowledge derives solely from experience. This places him with Francis Bacon, Thomas Hobbes, John Locke, and George Berkeley as a British Empiricist.

Hume argued that inductive reasoning and belief in causality cannot be justified rationally; instead, they result from custom and mental habit. We never actually perceive that one event causes another but only experience the "constant conjunction" of events. This problem of induction means that to draw any causal inferences from past experience, it is necessary to presuppose that the future will resemble the past, a presupposition which cannot itself be grounded in prior experience.

An opponent of philosophical rationalists, Hume held that passions rather than reason govern human behaviour, famously proclaiming that "Reason is, and ought only to be the slave of the passions."Hume was also a sentimentalist who held that ethics are based on emotion or sentiment rather than abstract moral principle. He maintained an early commitment to naturalistic explanations of moral phenomena and is usually taken to have first clearly expounded the is–ought problem, or the idea that a statement of fact alone can never give rise to a normative conclusion of what ought to be done.

Hume also denied that humans have an actual conception of the self, positing that we experience only a bundle of sensations, and that the self is nothing more than this bundle of causally-connected perceptions. Hume's compatibilist theory of free will takes causal determinism as fully compatible with human freedom. His views on philosophy of religion, including his rejection of miracles and the argument from design for God's existence, were especially controversial for their time.

Hume influenced utilitarianism, logical positivism, the philosophy of science, early analytic philosophy, cognitive science, theology, and many other fields and thinkers. Immanuel Kant credited Hume as the inspiration who had awakened him from his "dogmatic slumbers."

In [ ]:
Analyzer(dataFrame, 'Hume')

# Husserl

![](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8d/Edmund_Husserl_1910s.jpg/330px-Edmund_Husserl_1910s.jpg)

Edmund Gustav Albrecht Husserl ( April 1859 – 27 April 1938) was a German philosopher who established the school of phenomenology. In his early work, he elaborated critiques of historicism and of psychologism in logic based on analyses of intentionality. In his mature work, he sought to develop a systematic foundational science based on the so-called phenomenological reduction. Arguing that transcendental consciousness sets the limits of all possible knowledge, Husserl redefined phenomenology as a transcendental-idealist philosophy. Husserl's thought profoundly influenced 20th-century philosophy, and he remains a notable figure in contemporary philosophy and beyond.

Husserl studied mathematics, taught by Karl Weierstrass and Leo Königsberger, and philosophy taught by Franz Brentano and Carl Stumpf. He taught philosophy as a Privatdozent at Halle from 1887, then as professor, first at Göttingen from 1901, then at Freiburg from 1916 until he retired in 1928, after which he remained highly productive. In 1933, due to racial laws, having been born to a Jewish family, he was expelled from the library of the University of Freiburg, and months later resigned from the Deutsche Akademie. Following an illness, he died in Freiburg in 1938.

In [ ]:
Analyzer(dataFrame, 'Husserl')

# Kant

![](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f2/Kant_gemaelde_3.jpg/330px-Kant_gemaelde_3.jpg)

Immanuel Kant (22 April 1724 – 12 February 1804) was a German philosopher and one of the central Enlightenment thinkers. Kant's comprehensive and systematic works in epistemology, metaphysics, ethics, and aesthetics have made him one of the most influential figures in modern Western philosophy.

In his doctrine of transcendental idealism, Kant argued that space and time are mere "forms of intuition" which structure all experience, and therefore that while "things-in-themselves" exist and contribute to experience, they are nonetheless distinct from the objects of experience. From this it follows that the objects of experience are mere "appearances", and that the nature of things as they are in themselves is consequently unknowable to us. In an attempt to counter the skepticism he found in the writings of philosopher David Hume, he wrote the Critique of Pure Reason (1781/1787), one of his most well-known works. In it, he developed his theory of experience to answer the question of whether synthetic a priori knowledge is possible, which would in turn make it possible to determine the limits of metaphysical inquiry. Kant drew a parallel to the Copernican revolution in his proposal that the objects of the senses must conform to our spatial and temporal forms of intuition, and that we can consequently have a priori cognition of the objects of the senses.

Kant believed that reason is also the source of morality, and that aesthetics arise from a faculty of disinterested judgment. Kant's views continue to have a major influence on contemporary philosophy, especially the fields of epistemology, ethics, political theory, and post-modern aesthetics.He attempted to explain the relationship between reason and human experience and to move beyond what he believed to be the failures of traditional philosophy and metaphysics. He wanted to put an end to what he saw as an era of futile and speculative theories of human experience, while resisting the skepticism of thinkers such as Hume. He regarded himself as showing the way past the impasse between rationalists and empiricists, and is widely held to have synthesized both traditions in his thought.

Kant was an exponent of the idea that perpetual peace could be secured through universal democracy and international cooperation, and that perhaps this could be the culminating stage of world history.The nature of Kant's religious views continues to be the subject of scholarly dispute, with viewpoints ranging from the impression that he shifted from an early defense of an ontological argument for the existence of God to a principled agnosticism, to more critical treatments epitomized by Schopenhauer, who criticized the imperative form of Kantian ethics as "theological morals" and the "Mosaic Decalogue in disguise", and Nietzsche, who claimed that Kant had "theologian blood"and was merely a sophisticated apologist for traditional Christian faith. Beyond his religious views, Kant has also been criticized for the racism presented in some of his lesser-known papers, such as "On the Use of Teleological Principles in Philosophy" and "On the Different Races of Man". Although he was a proponent of scientific racism for much of his career, Kant's views on race changed significantly in the last decade of his life, and he ultimately rejected racial hierarchies and European colonialism in Perpetual Peace: A Philosophical Sketch (1795).

Kant published other important works on ethics, religion, law, aesthetics, astronomy, and history during his lifetime. These include the Universal Natural History (1755), the Critique of Practical Reason (1788), the Critique of Judgment (1790), Religion within the Bounds of Bare Reason (1793), and the Metaphysics of Morals (1797).

In [ ]:
Analyzer(dataFrame, 'Kant')

# Keynes
![](https://upload.wikimedia.org/wikipedia/commons/0/04/Keynes_1933.jpg)

John Maynard Keynes, 1st Baron Keynes CB FBA (5 June 1883 – 21 April 1946), was an English economist, whose ideas fundamentally changed the theory and practice of macroeconomics and the economic policies of governments. Originally trained in mathematics, he built on and greatly refined earlier work on the causes of business cycles, and was one of the most influential economists of the 20th century.His ideas are the basis for the school of thought known as Keynesian economics, and its various offshoots.

During the Great Depression of the 1930s, Keynes spearheaded a revolution in economic thinking, challenging the ideas of neoclassical economics that held that free markets would, in the short to medium term, automatically provide full employment, as long as workers were flexible in their wage demands. He argued that aggregate demand (total spending in the economy) determined the overall level of economic activity, and that inadequate aggregate demand could lead to prolonged periods of high unemployment. Keynes advocated the use of fiscal and monetary policies to mitigate the adverse effects of economic recessions and depressions. He detailed these ideas in his magnum opus, The General Theory of Employment, Interest and Money, published in 1936. By the late 1930s, leading Western economies had begun adopting Keynes's policy recommendations. Almost all capitalist governments had done so by the end of the two decades following Keynes's death in 1946. As a leader of the British delegation, Keynes participated in the design of the international economic institutions established after the end of World War II but was overruled by the American delegation on several aspects.

Keynes's influence started to wane in the 1970s, partly as a result of the stagflation that plagued the Anglo-American economies during that decade, and partly because of criticism of Keynesian policies by Milton Friedman and other monetarists,[6] who disputed the ability of government to favourably regulate the business cycle with fiscal policy.However, the advent of the global financial crisis of 2007–2008 sparked a resurgence in Keynesian thought. Keynesian economics provided the theoretical underpinning for economic policies undertaken in response to the financial crisis of 2007–2008 by President Barack Obama of the United States, Prime Minister Gordon Brown of the United Kingdom, and other heads of governments.

When Time magazine included Keynes among its Most Important People of the Century in 1999, it stated that "his radical idea that governments should spend money they don't have may have saved capitalism."[9] The Economist has described Keynes as "Britain's most famous 20th-century economist." In addition to being an economist, Keynes was also a civil servant, a director of the Bank of England, and a part of the Bloomsbury Group of intellectuals.


In [ ]:
Analyzer(dataFrame, 'Keynes')

# Kripke

![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d4/Kripke.JPG/330px-Kripke.JPG)

Saul Aaron Kripke (born November 13, 1940) is an American philosopher and logician in the analytic tradition. He is a Distinguished Professor of Philosophy at the Graduate Center of the City University of New York and emeritus professor at Princeton University. Since the 1960s, Kripke has been a central figure in a number of fields related to mathematical logic, modal logic, philosophy of language, philosophy of mathematics, metaphysics, epistemology, and recursion theory. Much of his work remains unpublished or exists only as tape recordings and privately circulated manuscripts.

Kripke has made influential and original contributions to logic, especially modal logic. His principal contribution is a semantics for modal logic involving possible worlds, now called Kripke semantics. He received the 2001 Schock Prize in Logic and Philosophy.

Kripke is also partly responsible for the revival of metaphysics after the decline of logical positivism, claiming necessity is a metaphysical notion distinct from the epistemic notion of a priori, and that there are necessary truths that are known a posteriori, such as that water is H2O. A 1970 Princeton lecture series, published in book form in 1980 as Naming and Necessity, is considered one of the most important philosophical works of the 20th century. It introduces the concept of names as rigid designators, true in every possible world, as contrasted with descriptions. It also contains Kripke's causal theory of reference, disputing the descriptivist theory found in Gottlob Frege's concept of sense and Bertrand Russell's theory of descriptions.

Kripke also gave an original reading of Ludwig Wittgenstein, known as "Kripkenstein", in his Wittgenstein on Rules and Private Language. The book contains his rule-following argument, a paradox for skepticism about meaning.

In [ ]:
Analyzer(dataFrame, 'Kripke')

# Leibniz

![](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8d/Christoph_Bernhard_Francke_-_Bildnis_des_Philosophen_Leibniz_%28ca._1695%29.jpg/330px-Christoph_Bernhard_Francke_-_Bildnis_des_Philosophen_Leibniz_%28ca._1695%29.jpg)

Gottfried Wilhelm (von) Leibniz ( 1 July 1646 – 14 November 1716) was a prominent German polymath and one of the most important logicians, mathematicians and natural philosophers of the Enlightenment. As a representative of the seventeenth-century tradition of rationalism, Leibniz developed, as his most prominent accomplishment, the ideas of differential and integral calculus, independently of Isaac Newton's contemporaneous developments.Mathematical works have consistently favored Leibniz's notation as the conventional expression of calculus. It was only in the 20th century that Leibniz's law of continuity and transcendental law of homogeneity found mathematical implementation (by means of non-standard analysis). He became one of the most prolific inventors in the field of mechanical calculators. While working on adding automatic multiplication and division to Pascal's calculator, he was the first to describe a pinwheel calculator in 1685 and invented the Leibniz wheel, used in the arithmometer, the first mass-produced mechanical calculator. He also refined the binary number system, which is the foundation of nearly all digital (electronic, solid-state, discrete logic) computers, including the Von Neumann machine, which is the standard design paradigm, or "computer architecture", followed from the second half of the 20th century, and into the 21st.

In philosophy, Leibniz is most noted for his optimism, i.e. his conclusion that our universe is, in a restricted sense, the best possible one that God could have created, an idea that was often lampooned by others such as Voltaire. Leibniz, along with René Descartes and Baruch Spinoza, was one of the three great 17th-century advocates of rationalism. The work of Leibniz anticipated modern logic and analytic philosophy, but his philosophy also assimilates elements of the scholastic tradition, notably that conclusions are produced by applying reason to first principles or prior definitions rather than to empirical evidence.

Leibniz made major contributions to physics and technology, and anticipated notions that surfaced much later in philosophy, probability theory, biology, medicine, geology, psychology, linguistics, and computer science. He wrote works on philosophy, politics, law, ethics, theology, history, and philology. Leibniz also contributed to the field of library science. While serving as overseer of the Wolfenbüttel library in Germany, he devised a cataloging system that would serve as a guide for many of Europe's largest libraries.Leibniz's contributions to this vast array of subjects were scattered in various learned journals, in tens of thousands of letters, and in unpublished manuscripts. He wrote in several languages, primarily in Latin, French and German but also in English, Italian and Dutch. There is no complete gathering of the writings of Leibniz translated into English.

In [ ]:
Analyzer(dataFrame, 'Leibniz')

# Lewis

![](https://upload.wikimedia.org/wikipedia/en/a/ad/David-K.-Lewis.jpg)

David Kellogg Lewis (September 28, 1941 – October 14, 2001) was an American philosopher who is widely regarded as one of the most important philosophers of the 20th century. Lewis taught briefly at UCLA and then at Princeton University from 1970 until his death. He is closely associated with Australia, whose philosophical community he visited almost annually for more than 30 years.

Lewis made significant contributions in philosophy of mind, philosophy of probability, epistemology, philosophical logic, aesthetics, philosophy of mathematics, philosophy of time and philosophy of science. In most of these fields he is considered among the most important figures of recent decades. But Lewis is most famous for his work in metaphysics, philosophy of language and semantics, in which his books On the Plurality of Worlds (1986) and Counterfactuals (1973) are considered classics. His works on the logic and semantics of counterfactual conditionals are broadly used by philosophers and linguists along with a competing account from Robert Stalnaker; together the Stalnaker-Lewis theory of counterfactuals has become perhaps the most pervasive and influential account of its type in the philosophical and linguistic literature. His metaphysics incorporated seminal contributions to quantified modal logic, the development of counterpart theory, counterfactual causation, and the position called "Humean supervenience". Most comprehensively in On the Plurality of Worlds, Lewis defended modal realism: the view that possible worlds exist as concrete entities in logical space, and that our world is one among many equally real possible ones.

In [ ]:
Analyzer(dataFrame, 'Lewis')

# Locke
![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b8/John_Locke.jpg/330px-John_Locke.jpg)

John Locke FRS (29 August 1632 – 28 October 1704) was an English philosopher and physician, widely regarded as one of the most influential of Enlightenment thinkers and commonly known as the "Father of Liberalism".Considered one of the first of the British empiricists, following the tradition of Sir Francis Bacon, Locke is equally important to social contract theory. His work greatly affected the development of epistemology and political philosophy. His writings influenced Voltaire and Jean-Jacques Rousseau, and many Scottish Enlightenment thinkers, as well as the American Revolutionaries. His contributions to classical republicanism and liberal theory are reflected in the United States Declaration of Independence.

Locke's theory of mind is often cited as the origin of modern conceptions of identity and the self, figuring prominently in the work of later philosophers such as Jean-Jacques Rousseau, David Hume, and Immanuel Kant. Locke was the first to define the self through a continuity of consciousness. He postulated that, at birth, the mind was a blank slate, or tabula rasa. Contrary to Cartesian philosophy based on pre-existing concepts, he maintained that we are born without innate ideas, and that knowledge is instead determined only by experience derived from sense perception, a concept now known as empiricism. Demonstrating the ideology of science in his observations, whereby something must be capable of being tested repeatedly and that nothing is exempt from being disproved, Locke stated that "whatever I write, as soon as I discover it not to be true, my hand shall be the forwardest to throw it into the fire". Such is one example of Locke's belief in empiricism.


In [ ]:
Analyzer(dataFrame, 'Locke')

# Malebranche

![](https://upload.wikimedia.org/wikipedia/commons/thumb/1/17/Nicolas_Malebranche.jpg/330px-Nicolas_Malebranche.jpg)

Nicolas Malebranche, Oratory of Jesus (August 1638 – 13 October 1715), was a French Oratorian[1] priest and rationalist philosopher. In his works, he sought to synthesize the thought of St. Augustine and Descartes, in order to demonstrate the active role of God in every aspect of the world. Malebranche is best known for his doctrines of vision in God, occasionalism and ontologism.

In [ ]:
Analyzer(dataFrame, 'Malebranche')

# Marx

![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d4/Karl_Marx_001.jpg/330px-Karl_Marx_001.jpg)

Karl Heinrich Marx FRSA (5 May 1818 – 14 March 1883) was a German philosopher, economist, historian, sociologist, political theorist, journalist and socialist revolutionary. Born in Trier, Germany, Marx studied law and philosophy at university. He married Jenny von Westphalen in 1843. Due to his political publications, Marx became stateless and lived in exile with his wife and children in London for decades, where he continued to develop his thought in collaboration with German thinker Friedrich Engels and publish his writings, researching in the reading room of the British Museum. His best-known titles are the 1848 pamphlet The Communist Manifesto and the three-volume Das Kapital (1867–1883). Marx's political and philosophical thought had enormous influence on subsequent intellectual, economic and political history. His name has been used as an adjective, a noun and a school of social theory.

Marx's critical theories about society, economics and politics, collectively understood as Marxism, hold that human societies develop through class conflict. In the capitalist mode of production, this manifests itself in the conflict between the ruling classes (known as the bourgeoisie) that control the means of production and the working classes (known as the proletariat) that enable these means by selling their labour power in return for wages.Employing a critical approach known as historical materialism, Marx predicted that capitalism produced internal tensions like previous socio-economic systems and that those would lead to its self-destruction and replacement by a new system known as the socialist mode of production. For Marx, class antagonisms under capitalism, owing in part to its instability and crisis-prone nature, would eventuate the working class' development of class consciousness, leading to their conquest of political power and eventually the establishment of a classless, communist society constituted by a free association of producers.Marx actively pressed for its implementation, arguing that the working class should carry out organised proletarian revolutionary action to topple capitalism and bring about socio-economic emancipation.

Marx has been described as one of the most influential figures in human history and his work has been both lauded and criticised. His work in economics laid the basis for much of the current understanding of labour and its relation to capital and subsequent economic thought. Many intellectuals, labour unions, artists and political parties worldwide have been influenced by Marx's work, with many modifying or adapting his ideas. Marx is typically cited as one of the principal architects of modern social science.

In [ ]:
Analyzer(dataFrame, 'Marx')

# Merleau-Ponty

![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/98/Maurice_Merleau-Ponty.jpg/330px-Maurice_Merleau-Ponty.jpg)

Maurice Jean Jacques Merleau-Ponty (14 March 1908 – 3 May 1961) was a French phenomenological philosopher, strongly influenced by Edmund Husserl and Martin Heidegger. The constitution of meaning in human experience was his main interest and he wrote on perception, art, politics, religion, psychology, psychoanalysis, language, nature, and history. He was the lead editor of Les Temps modernes, the leftist magazine he established with Jean-Paul Sartre and Simone de Beauvoir in 1945.

At the core of Merleau-Ponty's philosophy is a sustained argument for the foundational role perception plays in understanding the world as well as engaging with the world. Like the other major phenomenologists, Merleau-Ponty expressed his philosophical insights in writings on art, literature, linguistics, and politics. He was the only major phenomenologist of the first half of the twentieth century to engage extensively with the sciences and especially with Gestalt psychology. It is through this engagement that his writings became influential in the project of naturalizing phenomenology, in which phenomenologists use the results of psychology and cognitive science.

Merleau-Ponty emphasized the body as the primary site of knowing the world, a corrective to the long philosophical tradition of placing consciousness as the source of knowledge, and maintained that the body and that which it perceived could not be disentangled from each other. The articulation of the primacy of embodiment led him away from phenomenology towards what he was to call “indirect ontology” or the ontology of “the flesh of the world” (la chair du monde), seen in his final and incomplete work, The Visible and Invisible, and his last published essay, “Eye and Mind”.

In his earlier work, Merleau-Ponty supported Soviet communism while remaining critical of Soviet policies and Marxism in general, adopting a skeptical stance which he termed Western Marxism. His endorsement of the Soviet show trials and prison camps was published as Humanism and Terror in 1947, though he would later denounce Soviet terror as being counter to the purportedly humanist aims of the revolution.

In [ ]:
Analyzer(dataFrame, 'Merleau-Ponty')

# Moore

![](https://upload.wikimedia.org/wikipedia/commons/b/b9/1914_George_Edward_Moore_%28cropped%29.jpg)

George Edward Moore OM FBA (4 November 1873 – 24 October 1958), usually cited as G. E. Moore, was an English philosopher. He was, with Bertrand Russell, Ludwig Wittgenstein, and (before them) Gottlob Frege, one of the founders of analytic philosophy. Along with Russell, he led the turn away from idealism in British philosophy, and became well known for his advocacy of common sense concepts, his contributions to ethics, epistemology, and metaphysics, and "his exceptional personality and moral character".

Moore was Professor of Philosophy at the University of Cambridge, highly influential among (though not a member of) the Bloomsbury Group, and the editor of the influential journal Mind. He was elected a fellow of the British Academy in 1918. He was a member of the Cambridge Apostles, the intellectual secret society, from 1894 to 1901, and chairman of the Cambridge University Moral Sciences Club from 1912 to 1944. A humanist, he served as President of the British Ethical Union (now known as Humanists UK) from 1935 to 1936.

In [ ]:
Analyzer(dataFrame, 'Moore')

# Plato

![](https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Plato_Silanion_Musei_Capitolini_MC1377.jpg/330px-Plato_Silanion_Musei_Capitolini_MC1377.jpg)

Plato (428/427 or 424/423 – 348/347 BC) was an Athenian philosopher during the Classical period in Ancient Greece, founder of the Platonist school of thought and the Academy, the first institution of higher learning in the Western world.

He is widely considered the pivotal figure in the history of Ancient Greek and Western philosophy, along with his teacher, Socrates, and his most famous student, Aristotle.Plato has also often been cited as one of the founders of Western religion and spirituality. The so-called Neoplatonism of philosophers like Plotinus and Porphyry greatly influenced Christianity through Church Fathers such as Augustine. Alfred North Whitehead once noted: "the safest general characterization of the European philosophical tradition is that it consists of a series of footnotes to Plato."

Plato was the innovator of the written dialogue and dialectic forms in philosophy. Plato is also considered the founder of Western political philosophy. His most famous contribution is the theory of Forms known by pure reason, in which Plato presents a solution to the problem of universals known as Platonism (also ambiguously called either Platonic realism or Platonic idealism). He is also the namesake of Platonic love and the Platonic solids.

His own most decisive philosophical influences are usually thought to have been along with Socrates, the pre-Socratics Pythagoras, Heraclitus and Parmenides, although few of his predecessors' works remain extant and much of what we know about these figures today derives from Plato himself. Unlike the work of nearly all of his contemporaries, Plato's entire body of work is believed to have survived intact for over 2,400 years. Although their popularity has fluctuated over the years, Plato's works have never been without readers since the time they were written.

In [ ]:
Analyzer(dataFrame, 'Plato')

# Popper

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/43/Karl_Popper.jpg/330px-Karl_Popper.jpg)

Sir Karl Raimund Popper CH FBA FRS (28 July 1902 – 17 September 1994) was an Austrian-British[10] philosopher, academic and social commentator.

One of the 20th century's most influential philosophers of science, Popper is known for his rejection of the classical inductivist views on the scientific method in favour of empirical falsification. According to Popper, a theory in the empirical sciences can never be proven, but it can be falsified, meaning that it can (and should) be scrutinised with decisive experiments. Popper was opposed to the classical justificationist account of knowledge, which he replaced with critical rationalism, namely "the first non-justificational philosophy of criticism in the history of philosophy".

In political discourse, he is known for his vigorous defence of liberal democracy and the principles of social criticism that he believed made a flourishing open society possible. His political philosophy embraced ideas from major democratic political ideologies, including socialism/social democracy, libertarianism/classical liberalism and conservatism, and attempted to reconcile them.

In [ ]:
Analyzer(dataFrame, 'Popper')

# Quine
![](https://upload.wikimedia.org/wikipedia/commons/thumb/6/62/Wvq-passport-1975-400dpi-crop.jpg/330px-Wvq-passport-1975-400dpi-crop.jpg)

Willard Van Orman Quine (June 25, 1908 – December 25, 2000) was an American philosopher and logician in the analytic tradition, recognized as "one of the most influential philosophers of the twentieth century."From 1930 until his death 70 years later, Quine was continually affiliated with Harvard University in one way or another, first as a student, then as a professor. He filled the Edgar Pierce Chair of Philosophy at Harvard from 1956 to 1978.

Quine was a teacher of logic and set theory. Quine was famous for his position that first order logic is the only kind worthy of the name, and developed his own system of mathematics and set theory, known as New Foundations. In philosophy of mathematics, he and his Harvard colleague Hilary Putnam developed the "Quine–Putnam indispensability thesis," an argument for the reality of mathematical entities.However, he was the main proponent of the view that philosophy is not conceptual analysis, but continuous with science; the abstract branch of the empirical sciences. This led to his famous quip that "philosophy of science is philosophy enough." He led a "systematic attempt to understand science from within the resources of science itself"[13] and developed an influential naturalized epistemology that tried to provide "an improved scientific explanation of how we have developed elaborate scientific theories on the basis of meager sensory input." He also advocated ontological relativity in science, known as the Duhem–Quine thesis.

His major writings include the papers "On What There Is", which elucidated Bertrand Russell's theory of descriptions and contains Quine's famous dictum of ontological commitment, "To be is to be the value of a variable", and "Two Dogmas of Empiricism" (1951) which attacked the traditional analytic-synthetic distinction and reductionism, undermining the then-popular logical positivism, advocating instead a form of semantic holism. They also include the books The Web of Belief, which advocates a kind of coherentism, and Word and Object (1960), which further developed these positions and introduced Quine's famous indeterminacy of translation thesis, advocating a behaviorist theory of meaning.

A 2009 poll conducted among analytic philosophers named Quine as the fifth most important philosopher of the past two centuries.He won the first Schock Prize in Logic and Philosophy in 1993 for "his systematical and penetrating discussions of how learning of language and communication are based on socially available evidence and of the consequences of this for theories on knowledge and linguistic meaning." In 1996 he was awarded the Kyoto Prize in Arts and Philosophy for his "outstanding contributions to the progress of philosophy in the 20th century by proposing numerous theories based on keen insights in logic, epistemology, philosophy of science and philosophy of language."

In [ ]:
Analyzer(dataFrame, 'Quine')

# Ricardo

![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dc/Portrait_of_David_Ricardo_by_Thomas_Phillips.jpg/330px-Portrait_of_David_Ricardo_by_Thomas_Phillips.jpg)

David Ricardo (18 April 1772 – 11 September 1823) was a British political economist, one of the most influential of the classical economists along with Thomas Malthus, Adam Smith and James Mill. He was also a politician, and a member of the Parliament of Great Britain and Ireland.

In [ ]:
Analyzer(dataFrame, 'Ricardo')

# Russell

![](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a6/Bertrand_Russell_transparent_bg.png/225px-Bertrand_Russell_transparent_bg.png)

The aspects of Bertrand Russell's views on philosophy cover the changing viewpoints of philosopher and mathematician Bertrand Russell (1872–1970), from his early writings in 1896 until his death in February 1970.

In [ ]:
Analyzer(dataFrame, 'Russell')

# Smith

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/43/Adam_Smith_The_Muir_portrait.jpg/330px-Adam_Smith_The_Muir_portrait.jpg)

Adam Smith FRSA (16 June 1723 – 17 July 1790) was a Scottish economist, philosopher as well as a moral philosopher, a pioneer of political economy, and a key figure during the Scottish Enlightenment,also known as ''The Father of Economics'' or ''The Father of Capitalism''.Smith wrote two classic works, The Theory of Moral Sentiments (1759) and An Inquiry into the Nature and Causes of the Wealth of Nations (1776). The latter, often abbreviated as The Wealth of Nations, is considered his magnum opus and the first modern work of economics. In his work, Adam Smith introduced his theory of absolute advantage.

Smith studied social philosophy at the University of Glasgow and at Balliol College, Oxford, where he was one of the first students to benefit from scholarships set up by fellow Scot John Snell. After graduating, he delivered a successful series of public lectures at the University of Edinburgh, leading him to collaborate with David Hume during the Scottish Enlightenment. Smith obtained a professorship at Glasgow, teaching moral philosophy and during this time, wrote and published The Theory of Moral Sentiments. In his later life, he took a tutoring position that allowed him to travel throughout Europe, where he met other intellectual leaders of his day.

Smith laid the foundations of classical free market economic theory. The Wealth of Nations was a precursor to the modern academic discipline of economics. In this and other works, he developed the concept of division of labour and expounded upon how rational self-interest and competition can lead to economic prosperity. Smith was controversial in his own day and his general approach and writing style were often satirised by writers such as Horace Walpole.

In [ ]:
Analyzer(dataFrame, 'Smith')

# Spinoza

![](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Spinoza.jpg/330px-Spinoza.jpg)

Baruch (de) Spinoza (born Baruch Espinosa; later as an author and a correspondent Benedictus de Spinoza, anglicized to Benedict de Spinoza; 24 November 1632 – 21 February 1677) was a Dutch philosopher of Portuguese Sephardi origin. One of the early thinkers of the Enlightenment and modern biblical criticism, including modern conceptions of the self and the universe, he came to be considered one of the great rationalists of 17th-century philosophy.Inspired by the groundbreaking ideas of René Descartes, Spinoza became a leading philosophical figure of the Dutch Golden Age. Spinoza's given name, which means "Blessed", varies among different languages. In the Netherlands he used the Portuguese name Bento.In his Latin and Dutch works, he used Latin: Benedictus de Spinoza.

Spinoza was raised in the Portuguese-Jewish community in Amsterdam. He developed highly controversial ideas regarding the authenticity of the Hebrew Bible and the nature of the Divine. Jewish religious authorities issued a herem. against him, causing him to be effectively expelled and shunned by Jewish society at age 23, including by his own family. His books were later added to the Catholic Church's Index of Forbidden Books. He was frequently called an "atheist" by contemporaries, although nowhere in his work does Spinoza argue against the existence of God.

Spinoza lived an outwardly simple life as an optical lens grinder, collaborating on microscope and telescope lens designs with Constantijn and Christiaan Huygens. He turned down rewards and honours throughout his life, including prestigious teaching positions. He died at the age of 44 in 1677 from a lung illness, perhaps tuberculosis or silicosis exacerbated by the inhalation of fine glass dust while grinding lenses. He is buried in the Christian churchyard of Nieuwe Kerk in The Hague.

Spinoza's magnum opus, the Ethics, was published posthumously in the year of his death. The work opposed Descartes' philosophy of mind–body dualism and earned Spinoza recognition as one of Western philosophy's most important thinkers. In it, "Spinoza wrote the last indisputable Latin masterpiece, and one in which the refined conceptions of medieval philosophy are finally turned against themselves and destroyed entirely". Georg Wilhelm Friedrich Hegel said, "The fact is that Spinoza is made a testing-point in modern philosophy, so that it may really be said: You are either a Spinozist or not a philosopher at all." His philosophical accomplishments and moral character prompted Gilles Deleuze to name him "the 'prince' of philosophers."

In [ ]:
Analyzer(dataFrame, 'Spinoza')